In [34]:
import os
import numpy as np
import pandas as pd
import spacy
import plac
import random
from pathlib import Path
from tqdm import tqdm
import time
from datetime import datetime

# Functions

In [4]:
def write_data(data_list:list, file_name:str):
    
    """Takes list of data and file name, creates and writes the file"""
    
    with open(file_name, 'w') as file:
        for data in data_list:
            file.write(str(data)+'\n')

In [5]:
def load_train_data(file_name:str, label:str):
    
    """load train data from file file_name and assign label
    return training data as list for spacy"""
    
    if (label.upper()=='NONE'):
        data_list = []
        with open(file_name,'r') as file:
            for line in file.readlines():
                text = line.strip()
                data_list.append((text, {'entities': []}))
            file.close()
        return data_list
    
    else:
        data_list = []
        with open(file_name,'r') as file:
            for line in file.readlines():
                text = line.strip()
                data_list.append((text, {'entities': [(0,len(text),label.upper())]}))
            file.close()
        return data_list
    

In [6]:
def load_test_data(file:str, true_label:str):
    
    """load test data from file and return df"""
    
    test_data_ls=[]
    with open(file, 'r') as f:
        for line in f.readlines():
            test_data_ls.append(line.strip())
        f.close()
    df = pd.DataFrame({'data':test_data_ls, 'true_label':true_label.upper()})
    return df

# Train data

In [92]:
train_addr = load_train_data('train\\train_addr.txt','addr')
train_email = load_train_data('train\\train_email.txt','email')
train_none = load_train_data('train\\train_none.txt','none')
train_phone = load_train_data('train\\train_phone.txt','phone')

print(len(train_addr))
print(train_addr[:10])
print(len(train_email))
print(train_email[:10])
print(len(train_none))
print(train_none[:10])
print(len(train_phone))
print(train_phone[:10])

1700
[('2 RECTOR STREET NEW YORK NY 10006', {'entities': [(0, 33, 'ADDR')]}), ('115 S. LASALLE ST. CHICAGO', {'entities': [(0, 26, 'ADDR')]}), ('1818 State Route 3, Fulton NY 13069', {'entities': [(0, 35, 'ADDR')]}), ('P.O. Box 929 4189 Nunc Road, Lebanon KY 69409', {'entities': [(0, 45, 'ADDR')]}), ('3535 CANAL STREET NEW ORLEANS 70119', {'entities': [(0, 35, 'ADDR')]}), ('6140A Univ Drive, Huntsville AL 35806', {'entities': [(0, 37, 'ADDR')]}), ('5949 SHERRY LANE  SUITE 1500 DALLAS TX 75225', {'entities': [(0, 44, 'ADDR')]}), ('5050 POPLAR AVE., STE 1430 NJ 7044', {'entities': [(0, 34, 'ADDR')]}), ('589 5TH AVENUE NEW YORK NY 10017', {'entities': [(0, 32, 'ADDR')]}), ('701 Mcmeans Ave, Bay Minette AL 36507', {'entities': [(0, 37, 'ADDR')]})]
1300
[('Lauren_Campagne@gmail.com', {'entities': [(0, 25, 'EMAIL')]}), ('Edward_Briere@gmail.com', {'entities': [(0, 23, 'EMAIL')]}), ('Phillip_Dedmond@gmail.com', {'entities': [(0, 25, 'EMAIL')]}), ('Frank_Dent318@yahoo.com', {'entities': [(0, 2

In [93]:
TRAIN_DATA = train_addr + train_email + train_none + train_phone

print(len(TRAIN_DATA))
TRAIN_DATA[:10]

6700


[('2 RECTOR STREET NEW YORK NY 10006', {'entities': [(0, 33, 'ADDR')]}),
 ('115 S. LASALLE ST. CHICAGO', {'entities': [(0, 26, 'ADDR')]}),
 ('1818 State Route 3, Fulton NY 13069', {'entities': [(0, 35, 'ADDR')]}),
 ('P.O. Box 929 4189 Nunc Road, Lebanon KY 69409',
  {'entities': [(0, 45, 'ADDR')]}),
 ('3535 CANAL STREET NEW ORLEANS 70119', {'entities': [(0, 35, 'ADDR')]}),
 ('6140A Univ Drive, Huntsville AL 35806', {'entities': [(0, 37, 'ADDR')]}),
 ('5949 SHERRY LANE  SUITE 1500 DALLAS TX 75225',
  {'entities': [(0, 44, 'ADDR')]}),
 ('5050 POPLAR AVE., STE 1430 NJ 7044', {'entities': [(0, 34, 'ADDR')]}),
 ('589 5TH AVENUE NEW YORK NY 10017', {'entities': [(0, 32, 'ADDR')]}),
 ('701 Mcmeans Ave, Bay Minette AL 36507', {'entities': [(0, 37, 'ADDR')]})]

In [94]:
TRAIN_DATA[-10:]

[('399.670-0945', {'entities': [(0, 12, 'PHONE')]}),
 ('+3(558) 904.1728', {'entities': [(0, 16, 'PHONE')]}),
 ('+1 995-765-4530', {'entities': [(0, 15, 'PHONE')]}),
 ('551 978 8144', {'entities': [(0, 12, 'PHONE')]}),
 ('+7.(693).480 4334', {'entities': [(0, 17, 'PHONE')]}),
 ('+2.(294)-415-5499', {'entities': [(0, 17, 'PHONE')]}),
 ('+16102347339', {'entities': [(0, 12, 'PHONE')]}),
 ('+3 (768).819.4668', {'entities': [(0, 17, 'PHONE')]}),
 ('+0-(381)-635.0731', {'entities': [(0, 17, 'PHONE')]}),
 ('+1 (945) 879-2933', {'entities': [(0, 17, 'PHONE')]})]

# Model

### load

In [95]:
# load model
nlp = spacy.load('en_core_web_sm')  
print("Model loaded")

#set up the pipeline

if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Model loaded


In [96]:
nlp.pipe_names

['tagger', 'parser', 'ner']

In [97]:
ner

### train

In [98]:
n_iter = 50
drop = 0.5

for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    with open('log\\log_train_'+str(datetime.now().strftime("%Y%m%d_%H%M%S"))+'.txt','w') as logfile:
        logfile.write('Training points: '+str(len(TRAIN_DATA))+'\n')
        logfile.write('Iterations: '+str(n_iter)+ '\n')
        logfile.write('drop: '+ str(drop) + '\n\n')
        start_outer = time.time()
        
        for itn in range(n_iter):
            start_inner = time.time()
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in tqdm(TRAIN_DATA):
                nlp.update(
                    [text],  
                    [annotations],  
                    drop=drop,  
                    sgd=optimizer,
                    losses=losses)
            
            stop_inner = time.time()
            print(str(itn) +'\t'+ 't:'+str(stop_inner - start_inner) +'\t'+ str(losses))
            logfile.write(str(itn)+'\t'+'t: '+str(stop_inner - start_inner)+'\t'+str(losses)+'\n')
        
        stop_outer = time.time()
        print(f'\n\n Total Training time: {stop_outer-start_outer} seconds')
        logfile.write('\n\nTotal Training time: '+ str(stop_outer - start_outer)+ ' s')
        logfile.close()


  0%|                                                                                 | 2/6700 [00:00<09:36, 11.61it/s]

t:527.3010256290436	{'ner': 17638.887911547314}


  0%|                                                                                 | 2/6700 [00:00<09:59, 11.17it/s]

t:548.7311816215515	{'ner': 16149.14939398244}


  0%|                                                                                 | 2/6700 [00:00<07:51, 14.21it/s]

...

t:566.9180226325989	{'ner': 15492.81969218689}


  0%|                                                                                 | 2/6700 [00:00<09:40, 11.54it/s]

t:564.7556402683258	{'ner': 15611.32711907463}


100%|██████████████████████████████████████████████████████████████████████████████| 6700/6700 [09:25<00:00, 11.84it/s]

t:565.9345936775208	{'ner': 15745.414783854812}


 Total Training time: 83996.78734016418 seconds


### Save

In [99]:
# save model
output_dir = 'model_spacy_email_phone_addr_update_pretrained'

nlp.to_disk(output_dir)
print("Saved model to", output_dir)

Saved model to model_spacy_email_phone_addr_update_pretrained


### test - general

In [100]:
doc = nlp("amsf.as@yahoo.com")
print([(X.text, X.label_) for X in doc.ents])

[('amsf.as@yahoo.com', 'EMAIL')]


In [101]:
doc = nlp("+16102899411")
print([(X.text, X.label_) for X in doc.ents])

[('+16102899411', 'PHONE')]


In [102]:
doc = nlp("302 Park street")
print([(X.text, X.label_) for X in doc.ents])

[('302 Park street', 'ADDR')]


In [103]:
doc = nlp("909 FANNIN TWO HOUSTON CENTER, SUITE 2310")
print([(X.text, X.label_) for X in doc.ents])

[('909 FANNIN TWO HOUSTON CENTER, SUITE 2310', 'ADDR')]


In [104]:
doc = nlp("+159901011")
print([(X.text, X.label_) for X in doc.ents])

[('+159901011', 'PHONE')]


In [105]:
doc = nlp("DUKE ENERGY TRADING AND MARKETING")
print([(X.text, X.label_) for X in doc.ents])

[]


In [106]:
doc = nlp("New york")
print([(X.text, X.label_) for X in doc.ents])

[]


In [107]:
doc = nlp("LA")
print([(X.text, X.label_) for X in doc.ents])

[]


In [108]:
doc = nlp("495668")
print([(X.text, X.label_) for X in doc.ents])

[]


In [127]:
doc = nlp("Microsoft")
print([(X.text, X.label_) for X in doc.ents])

[]


In [129]:
doc = nlp("https://www.google.com")
print([(X.text, X.label_) for X in doc.ents])

[]


In [130]:
doc = nlp("english")
print([(X.text, X.label_) for X in doc.ents])

[]
